In [16]:
import numpy as np

In [17]:
# helper functions
def get_r(closing_price):
    rs = []
    for i in range(1, len(closing_price)):
        q = (closing_price[i] - closing_price[i-1])/closing_price[i-1]
        r = np.log(1 + q)
        rs.append(r)
    return rs
    
def cor_ij(ri, rj):
    inner_ij = np.mean(np.multiply(ri, rj))
    inner_ii = np.mean(np.multiply(ri, ri))
    inner_jj = np.mean(np.multiply(rj, rj))
    
    cor = (inner_ij - np.mean(ri) * np.mean(rj))/(np.sqrt((inner_ii-(np.mean(ri))**2)*(inner_jj - (np.mean(rj))**2)))
    return cor
    

In [18]:
import os
import pandas as pd

dirs = os.path.abspath("finance_data/data")
file_to_r = {}

for file in os.listdir(dirs):
    if (file[-4:] != '.csv'):
        continue
    file_name = file[:-4]
    file_path = dirs + '/' + file
    df = pd.read_csv(file_path)
    dates = df['Date']
    closing_prices = df['Close']
    rs = []
    if len(closing_prices) != 765:
        continue
    rs = get_r(closing_prices)
        
    file_to_r[file_name] = rs

In [19]:
print(len(file_to_r))

494


In [22]:
file_out = open(r"weight_day.txt", "w")
file_names = list(file_to_r.keys())

for i in range(len(file_names)):
    for j in range(i+1, len(file_names)):
        r_i = np.array(file_to_r[file_names[i]])
        r_j = np.array(file_to_r[file_names[j]])
        ro_i_j = cor_ij(r_i, r_j)
        w_i_j = np.sqrt(2*(1-ro_i_j))
        if w_i_j != 0:
            file_out.write('{} {} {}\n'.format(file_names[i], file_names[j], w_i_j))

file_out.close()

In [23]:
dirs = os.path.abspath("finance_data/data")
file_to_r_week = {}

for file in os.listdir(dirs):
    if (file[-4:] != '.csv'):
        continue
    file_name = file[:-4]
    file_path = dirs + '/' + file
    df = pd.read_csv(file_path)
    dates = df['Date']
    closing_prices = df['Close']
    rs = []
    if len(closing_prices) != 765:
        continue
    for i in range(1, 765):
        if pd.Timestamp(dates[i]).weekday() == 0:
            q = (closing_prices[i] - closing_prices[i-1])/closing_prices[i-1]
            r = np.log(1+q)
            rs.append(r)
        
    file_to_r_week[file_name] = rs

In [24]:
file_out = open(r"weight_week.txt", "w")
file_names = list(file_to_r_week.keys())

for i in range(len(file_names)):
    for j in range(i+1, len(file_names)):
        r_i = np.array(file_to_r_week[file_names[i]])
        r_j = np.array(file_to_r_week[file_names[j]])
        ro_i_j = cor_ij(r_i, r_j)
        w_i_j = np.sqrt(2*(1-ro_i_j))
        if w_i_j != 0:
            file_out.write('{} {} {}\n'.format(file_names[i], file_names[j], w_i_j))

file_out.close()